# Определение стоимости автомобилей

## Превью

В качестве входных данных в проекте представлена таблица с 15 признаками и она же содержит еще один целевой признак. Оценочно некотрые признаки должны иметь формат даты, а многие являются категориальными. Предстоит разобраться, что делать с такими фичами и в каком виде их лучше передавать в модель. Также необходимо произвести предобработку данных, удалить выбросы, заменить значения ячеек nan, либо удалить строки с ними. Ну и про возможные неинформтаивные признаки тоже не забыть. В дальнейшем необходимо проверить качество и время работы модели на основе градиентного бустинга из библиотеки LGBM, и для сравнения других, более простых моделей. В качестве них возьмем линейную регрессию и случайный лес. Для моделей стоит попробовать различные гиперпараметры, в целях улучшения качества предсказания, при этом время обучения не должно быть слишком большим, поскольку это является одним из условий закзчика.

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder

In [2]:
%%time 
auto = pd.read_csv('/datasets/autos.csv')
auto.info()
display(auto.head(10))

print('-----------------------------------------------')
print('Количество бесплатных автомобилей', auto[auto['Price'] == 0]['Price'].count())
print('-----------------------------------------------')

#замена даты на месяц
auto['DateCrawled'] = auto['DateCrawled'].map(lambda x: x[6:7])
#auto['DateCrawled']=pd.to_datetime(auto['DateCrawled'], format='%Y-%m-%d')
auto['DateCreated'] = auto['DateCreated'].map(lambda x: x[6:7])
#auto['DateCreated']=pd.to_datetime(auto['DateCreated'], format='%Y-%m-%d')
auto['LastSeen'] = auto['LastSeen'].map(lambda x: x[6:7])
#auto['LastSeen']=pd.to_datetime(auto['LastSeen'], format='%Y-%m-%d')
#sns_plot = sns.distplot(auto[auto['Price'] > 50]['Price'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


-----------------------------------------------
Количество бесплатных автомобилей 10772
-----------------------------------------------
CPU times: user 1.09 s, sys: 136 ms, total: 1.22 s
Wall time: 1.25 s


In [3]:
%%time 
#Анализ влияния фич 'RegistrationMonth' и 'NumberOfPictures' на цену авто
print('В каких годах машины приобретались в месяце 0 \r\n', auto[auto['RegistrationMonth'] == 0]['RegistrationYear'].value_counts())
print('Правда ли фоторафии отсутсвуют во всех объявлениях \r\n', auto['NumberOfPictures'].value_counts())
print('Средняя цена по месяцам регистрации автомобиля')
auto_pivot = auto.pivot_table(index = 'RegistrationMonth', columns = 'NumberOfPictures', values = 'Price', aggfunc = 'mean')
display(auto_pivot)
display(auto_pivot.describe())

В каких годах машины приобретались в месяце 0 
 2000    4871
2016    2753
2017    2717
1999    2427
1995    2308
        ... 
2290       1
3700       1
1941       1
5300       1
1600       1
Name: RegistrationYear, Length: 127, dtype: int64
Правда ли фоторафии отсутсвуют во всех объявлениях 
 0    354369
Name: NumberOfPictures, dtype: int64
Средняя цена по месяцам регистрации автомобиля


NumberOfPictures,0
RegistrationMonth,
0,1907.968757
1,4670.885783
2,4581.080218
3,4585.424228
4,4744.656577
5,4623.699345
6,4706.366859
7,4798.471944
8,4665.058691


NumberOfPictures,0
count,13.000000
mean,4498.886689
std,785.683856
min,1907.968757
25%,4615.490469
50%,4670.885783
75%,4798.471944
max,4904.757422


CPU times: user 53.4 ms, sys: 22.2 ms, total: 75.6 ms
Wall time: 85.2 ms


In [4]:
%%time 
#Анализ влияния фичи 'DateCreated' на цену авто
#auto['date_created_year'] = pd.DatetimeIndex(auto['DateCreated']).year
#display(auto['date_created_year'].value_counts())
#auto_pivot_created_year = auto.pivot_table(index = 'date_created_year', columns = 'NumberOfPictures', values = 'Price', aggfunc = 'mean')
#display(auto_pivot_created_year)
#display(auto_pivot_created_year.describe())

#Анализ фичи 'PostalCode'
print('Количество уникальных значений индексов \r\n',auto['PostalCode'].unique().shape[0])

#Анализ фичи 'Power'
print('Автомобили с разными мощностями \r\n', auto['Power'].value_counts().sort_values())

#Анализ фичи 'Model'
print('Количество пропущеных значений в поле "Model" \r\n', auto[auto['Model'].isnull()]['DateCrawled'].count())


#Удаление неинформативных фич и записей, выбросов
auto = auto.drop(['NumberOfPictures','RegistrationMonth', 'PostalCode'], axis = 1)
auto = auto[auto['Price'] != 0]
#auto = auto[auto['date_created_year'] == 2016]
#auto = auto.drop(['date_created_year'], axis = 1)
auto = auto[auto['RegistrationYear'] > 1900]
auto = auto[auto['Power'] < 2000]
auto = auto[auto['Power'] != 0].reset_index(drop = True)
auto = auto.fillna('n/d')
auto.info()
display(auto.head(20))



Количество уникальных значений индексов 
 8143
Автомобили с разными мощностями 
 1158         1
7529         1
1771         1
10000        1
604          1
         ...  
101      13298
150      14590
60       15897
75       24023
0        40225
Name: Power, Length: 712, dtype: int64
Количество пропущеных значений в поле "Model" 
 19705
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307106 entries, 0 to 307105
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   DateCrawled       307106 non-null  object
 1   Price             307106 non-null  int64 
 2   VehicleType       307106 non-null  object
 3   RegistrationYear  307106 non-null  int64 
 4   Gearbox           307106 non-null  object
 5   Power             307106 non-null  int64 
 6   Model             307106 non-null  object
 7   Kilometer         307106 non-null  int64 
 8   FuelType          307106 non-null  object
 9   Brand             307106 non

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,Repaired,DateCreated,LastSeen
0,3,18300,coupe,2011,manual,190,n/d,125000,gasoline,audi,yes,3,4
1,3,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,n/d,3,4
2,3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no,3,3
3,3,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no,3,4
4,4,650,sedan,1995,manual,102,3er,150000,petrol,bmw,yes,4,4
5,4,2200,convertible,2004,manual,109,2_reihe,150000,petrol,peugeot,no,4,4
6,4,14500,bus,2014,manual,125,c_max,30000,petrol,ford,n/d,4,4
7,3,999,small,1998,manual,101,golf,150000,n/d,volkswagen,n/d,3,3
8,3,2000,sedan,2004,manual,105,3_reihe,150000,petrol,mazda,no,3,4
9,4,2799,wagon,2005,manual,140,passat,150000,gasoline,volkswagen,yes,4,4


CPU times: user 381 ms, sys: 87.4 ms, total: 469 ms
Wall time: 487 ms


### Выводы по подготовке данных

В ходе анализа и подготовки данных бросилось в глаза большое количество признаков с <b>пропущеными значениями</b>. Все их решено было заменить на значение <b>"n/d"</b>- нет данных, поскольку привязать тип кузова, тип коробки передач, модель и тип топлива по марке автомобиля довльно сложно (существует слишком много различных комплектаций), а побывал ли автомобиль в ремонте вообще не зависит от каких-либо других параметров. Также удалось выяснить, что данные взяты <b>за 2016 год</b>, лишь 26 значений относятся к 2014 и 2015 году, поэтому их решено удалить, как выбросы. Так как все объявления распределены в течение 1 года, то можно заменить </b>дату на номер месяца</b>, в целях уменьшения количества признаков (при переводе из формата date в int для удобства кодирования и дальнейшей работы модели). Также проведен анализ распределения средней цены в зависимости от месяца продажи. Значительных изменений не отмечено, <b>стоимость варьируется в диапозоне 4600 - 4900 евро</b>. Выбивающимся значением является средний показатель 1907 евро в <b>месяце с индексом 0</b>. В идеале такие проблемы необходимо решать с заказчиком, но поскольку это значение меньше 1 квантиля, решено их удалить. Удалены записи с полем <b>'Price' равным 0</b>. Возможно кто то просто хочет отдать автомобиль, но таких записей всего 3% процента от общего количества. Также не берутся для работы модели записи со значением поля <b>"Power" равным 0 и больше 2000</b> (выбросы). Не понятно, как могут повлиять <b>индекс</b> пользователя на цену автомобиля (в течение 1 года). Поэтому данная фича также была удалена из конечного датасета признаков для уменьшения времени обучения модели. Произведена процедура стандартизации признаков и порядкового кодирования для категориальных признаков. В итоге получено <b>12 признаков</b>.     

## Обучение моделей

In [5]:
%%time 
target = auto['Price']
features = auto.drop('Price', axis = 1)
encoder = OrdinalEncoder()
coder_columns = ['RegistrationYear', 'VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired', 'DateCrawled', 'DateCreated', 'LastSeen']
features[coder_columns] = pd.DataFrame(encoder.fit_transform(features[coder_columns]), columns=coder_columns)

feature_train, feature_test, target_train, target_test = train_test_split(features, target, test_size = 0.25, random_state = 12345)
scaler_columns = ['Power', 'Kilometer']
scaler = StandardScaler()
scaler.fit(feature_train[scaler_columns])
feature_train[scaler_columns] = scaler.transform(feature_train[scaler_columns])
feature_test[scaler_columns] = scaler.transform(feature_test[scaler_columns])
display(feature_train.head(10))

smape_scorerrr = make_scorer(mean_squared_error, greater_is_better=False)
model_2 = LinearRegression()



<timed exec>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
<timed exec>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

,DateCrawled,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,Repaired,DateCreated,LastSeen
278376,0.0,6.0,83.0,1.0,-0.827675,177.0,-0.092427,7.0,9.0,1.0,2.0,0.0
7295,0.0,8.0,78.0,0.0,1.255830,29.0,0.587358,7.0,1.0,1.0,2.0,1.0
142954,0.0,8.0,69.0,1.0,0.237935,165.0,0.587358,7.0,24.0,1.0,2.0,0.0
175795,0.0,5.0,74.0,1.0,-0.732247,120.0,0.587358,7.0,30.0,0.0,2.0,0.0
148632,0.0,8.0,80.0,0.0,0.667359,11.0,0.587358,2.0,2.0,1.0,2.0,0.0
163778,0.0,6.0,71.0,1.0,-0.891293,83.0,0.587358,7.0,24.0,1.0,2.0,1.0
4546,0.0,8.0,71.0,1.0,0.460599,11.0,0.587358,7.0,2.0,1.0,2.0,1.0
74007,0.0,2.0,84.0,1.0,0.556027,167.0,-2.131784,7.0,25.0,1.0,2.0,1.0
222340,0.0,1.0,76.0,1.0,-0.016539,159.0,0.587358,2.0,27.0,2.0,2.0,0.0
193511,0.0,5.0,70.0,1.0,-0.732247,116.0,0.587358,7.0,38.0,2.0,2.0,1.0


CPU times: user 1.03 s, sys: 105 ms, total: 1.13 s
Wall time: 1.15 s


In [6]:
%%time
#Обучение линейной регрессии
model_2.fit(feature_train, target_train)


CPU times: user 92.7 ms, sys: 50.6 ms, total: 143 ms
Wall time: 136 ms


LinearRegression()

In [7]:
%%time
#Предсказание линейной регрессии на тренировочной выборке
predicted_train = model_2.predict(feature_train)
mse_lr_train = mean_squared_error(target_train, predicted_train)
print("Для логистической регрессии лучшее значение rmse для тренировочных данных", (abs(mse_lr_train)**0.5))      
print("---------------------------------------------")

Для логистической регрессии лучшее значение rmse для тренировочных данных 3228.5565822461463
---------------------------------------------
CPU times: user 19.8 ms, sys: 40.6 ms, total: 60.3 ms
Wall time: 85.1 ms


In [8]:
%%time
#Предсказание линейной регрессии на тестовой выборке
predicted = model_2.predict(feature_test)
mse_lr = mean_squared_error(target_test, predicted) 
print("Для логистической регрессии лучшее значение rmse для тестовых данных", (abs(mse_lr)**0.5))      
print("---------------------------------------------")

Для логистической регрессии лучшее значение rmse для тестовых данных 3255.997043564626
---------------------------------------------
CPU times: user 11.8 ms, sys: 17.9 ms, total: 29.7 ms
Wall time: 8.87 ms


In [9]:
%%time 
# Подбор параметров случайного леса
model_1 = RandomForestRegressor()
grid = GridSearchCV(estimator = model_1,
                                param_grid={'n_estimators': [i for i in range(1, 5)], 'max_depth' : [c for c in range(1,5)],
                                           'random_state': [12345]},
                                scoring = smape_scorerrr, cv = 5) 
grid.fit(feature_train, target_train)
print(grid.best_params_)
print("Для случайного леса наилучшее значение rmse", ((abs(grid.best_score_))**0.5),'достигается при', grid.best_params_)
print("---------------------------------------------")

{'max_depth': 4, 'n_estimators': 4, 'random_state': 12345}
Для случайного леса наилучшее значение rmse 2669.4963721426093 достигается при {'max_depth': 4, 'n_estimators': 4, 'random_state': 12345}
---------------------------------------------
CPU times: user 19.7 s, sys: 152 ms, total: 19.9 s
Wall time: 20 s


In [10]:
%%time 
#Обучение случайного леса
model_best_forest = RandomForestRegressor(max_depth = 4, n_estimators = 4, random_state = 12345)
model_best_forest.fit(feature_train, target_train)

CPU times: user 678 ms, sys: 66 µs, total: 678 ms
Wall time: 684 ms


RandomForestRegressor(max_depth=4, n_estimators=4, random_state=12345)

In [11]:
%%time 
#Предсказание случайного леса на тестовых данных
predicted = model_best_forest.predict(feature_test)
mse_rf = mean_squared_error(target_test, predicted)
print("Для случайного леса по тестовым данным значение rmse", (mse_rf)**0.5)

Для случайного леса по тестовым данным значение rmse 2685.0904002429556
CPU times: user 17 ms, sys: 51 µs, total: 17.1 ms
Wall time: 15.2 ms


In [16]:
%%time 
model = lgb.LGBMRegressor()
# Подбор параметров модели LGBM
features = auto.drop('Price', axis = 1)
display(features.head())
coder_columns = ['RegistrationYear', 'VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired', 'DateCrawled', 'DateCreated', 'LastSeen']
features[coder_columns] = features[coder_columns].astype('category')
display(features.head())
feature_train, feature_test, target_train, target_test = train_test_split(features, target, test_size = 0.25, random_state = 12345)
scaler_columns = ['Power', 'Kilometer']
scaler = StandardScaler()
scaler.fit(feature_train[scaler_columns])
feature_train[scaler_columns] = scaler.transform(feature_train[scaler_columns])
feature_test[scaler_columns] = scaler.transform(feature_test[scaler_columns])


#grid = GridSearchCV(estimator = model,
#                                param_grid={'n_estimators': [i for i in range(500, 1501, 100)], 'max_depth' : [c for c in range(3, 5)],
#                                            'num_leaves': [l for l in range(4, 25, 4)], 'learning_rate': [0.03, 0.1, 0.5], 'random_state': [12345]},
#                                scoring = smape_scorerrr, cv = 5)
#'num_leaves': [l for l in range(4, 9, 4)]
grid = GridSearchCV(estimator = model,
                                param_grid={'n_estimators': [i for i in range(500, 701, 100)], 'max_depth' : [3,5],
                                            'learning_rate': [0.04, 0.1], 'random_state': [12345]},
                                scoring = smape_scorerrr, cv = 5)
grid.fit(feature_train, target_train)

print("Для lgbm-модели лучшее значение rsme", (abs(grid.best_score_)**0.5))
print("Для lgbm-модели значение параметров", grid.best_params_)
print("---------------------------------------------")



,DateCrawled,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,Repaired,DateCreated,LastSeen
0,3,coupe,2011,manual,190,n/d,125000,gasoline,audi,yes,3,4
1,3,suv,2004,auto,163,grand,125000,gasoline,jeep,n/d,3,4
2,3,small,2001,manual,75,golf,150000,petrol,volkswagen,no,3,3
3,3,small,2008,manual,69,fabia,90000,gasoline,skoda,no,3,4
4,4,sedan,1995,manual,102,3er,150000,petrol,bmw,yes,4,4


,DateCrawled,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,Repaired,DateCreated,LastSeen
0,3,coupe,2011,manual,190,n/d,125000,gasoline,audi,yes,3,4
1,3,suv,2004,auto,163,grand,125000,gasoline,jeep,n/d,3,4
2,3,small,2001,manual,75,golf,150000,petrol,volkswagen,no,3,3
3,3,small,2008,manual,69,fabia,90000,gasoline,skoda,no,3,4
4,4,sedan,1995,manual,102,3er,150000,petrol,bmw,yes,4,4


<timed exec>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
<timed exec>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

Для lgbm-модели лучшее значение rsme 1552.4573154741472
Для lgbm-модели значение параметров {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 700, 'random_state': 12345}
---------------------------------------------
CPU times: user 1h 22min 10s, sys: 31.2 s, total: 1h 22min 42s
Wall time: 1h 23min 34s


In [13]:
%%time
#Обучение модели LGBM
model_1 = lgb.LGBMRegressor(n_estimators = 700, max_depth = 5, learning_rate = 0.1, random_state = 12345)
model_1.fit(feature_train, target_train)

CPU times: user 5min 13s, sys: 2.13 s, total: 5min 15s
Wall time: 5min 18s


LGBMRegressor(max_depth=5, n_estimators=700, random_state=12345)

In [18]:
%%time
features = auto.drop('Price', axis = 1)
display(features.head())
coder_columns = ['RegistrationYear', 'VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired', 'DateCrawled', 'DateCreated', 'LastSeen']
features[coder_columns] = features[coder_columns].astype('category')
display(features.head())
feature_train, feature_test, target_train, target_test = train_test_split(features, target, test_size = 0.25, random_state = 12345)
scaler_columns = ['Power', 'Kilometer']
scaler = StandardScaler()
scaler.fit(feature_train[scaler_columns])
feature_train[scaler_columns] = scaler.transform(feature_train[scaler_columns])
feature_test[scaler_columns] = scaler.transform(feature_test[scaler_columns])

,DateCrawled,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,Repaired,DateCreated,LastSeen
0,3,coupe,2011,manual,190,n/d,125000,gasoline,audi,yes,3,4
1,3,suv,2004,auto,163,grand,125000,gasoline,jeep,n/d,3,4
2,3,small,2001,manual,75,golf,150000,petrol,volkswagen,no,3,3
3,3,small,2008,manual,69,fabia,90000,gasoline,skoda,no,3,4
4,4,sedan,1995,manual,102,3er,150000,petrol,bmw,yes,4,4


,DateCrawled,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,Repaired,DateCreated,LastSeen
0,3,coupe,2011,manual,190,n/d,125000,gasoline,audi,yes,3,4
1,3,suv,2004,auto,163,grand,125000,gasoline,jeep,n/d,3,4
2,3,small,2001,manual,75,golf,150000,petrol,volkswagen,no,3,3
3,3,small,2008,manual,69,fabia,90000,gasoline,skoda,no,3,4
4,4,sedan,1995,manual,102,3er,150000,petrol,bmw,yes,4,4


CPU times: user 378 ms, sys: 11.5 ms, total: 390 ms
Wall time: 399 ms


<timed exec>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
<timed exec>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [15]:
%%time
#Предсказание модели LGBM на тестовых данных
predicted = model_1.predict(feature_test)
mse_lgbm = mean_squared_error(target_test, predicted)
print("Для градиентного бустинга по тестовым данным значение rmse", (mse_lgbm)**0.5)
print("---------------------------------------------")

Для градиентного бустинга по тестовым данным значение rmse 1601.396880653598
---------------------------------------------
CPU times: user 4.87 s, sys: 0 ns, total: 4.87 s
Wall time: 4.84 s


## Анализ моделей

    Модель LGBM на основе градиентного бустинга затрачивет больше вренени на обучение (40 с), по сравнению с алгоритмом линейной регрессии (73 мс) и случайным лесом (637 мс). Она работает более чем в 63 раза дольше, по сравнению с лесом, и превосходит по времени линейную регрессию в 537 раз. В свою очередь качество предсказания моделей также значительно отличается уже в пользу градиентного бустинга. Если rsme для линейной регрессии 3255, то для случайного леса с лучшими параметрами 2685. А для градиентного бустинга 1598 (в 2 раза лучше линейной регрессии и на 40% лучше леса). Общее качество предсказания моделей оставляет желать лучшего, но для сравнения моделей данных достаточно. Время предсказания моделей 14,2 мс и 7,67 мс для леса и линейной регрессии отличается также значительно от времени предсказания lgbm модели 9,14 с (в 643 и 1190 раз соответственно). Значение функции потерь rsme отличается аналогично тренировочным данным. 

### Общий вывод

    В ходе выполнения проекта на первом этаме производилась предобработка данных и оценка возможности влияния различных фичей на конечный результат. Поскольку в качестве модели планировалось использовать алгоритм обучения на основе градиентного бустинга отбор неинформативных фичей и записей (выбросов) имеет очень важное влияние на время работы алгоритма. Далее осуществена процедура преобразования количественных данных (масштабирование) и категориальных признаков (порядковое кодирование). В качестве моделей рассматривались алгоритмы линейной регрессии, случайного леса и LGBM c незначмительным выбором параметров. Итогом стало более долгое обучение модели на основе градиентного бустинга. Но если сравнивать качество оценивания стоимости авто, то по казатели LGBM-модели не идут ни в какое сравнение с другими моделями и значительно их превосходят. 

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнена загрузка и подготовка данных
- [ ]  Выполнено обучение моделей
- [ ]  Есть анализ скорости работы и качества моделей